# Import Required Libraries
Import the necessary libraries, including LeagueHistoryLoader.

In [4]:
# LeagueHistoryLoader for loading the league history data
from src.fpl_load import LeagueHistoryLoader

# Setting the league id
league_id = 665568

# Creating an instance of LeagueHistoryLoader
history_loader = LeagueHistoryLoader(league_id)

In [9]:
# Loading the league history data
league_data = history_loader.get_data()

df = league_data
import duckdb

# Convert DataFrame to DuckDB
duckdb_df = duckdb.from_df(df)

# Answer Initial Questions
Use the loaded data to answer the questions found in the Initial Questions Answered section in README.md.

In [6]:
df

,event,event_points,cumulative_points,fpl_event_rank,fpl_event_rank_sort,overall_rank,percentile_rank,bank,team_value,event_transfers,...,points_on_bench,entry,id,event_total,player_name,league_rank,last_rank,rank_sort,standings_total,entry_name
0,1,80,80,710070,784384,710070,NaN,0,1000,0,...,11,3110982,21377033,59,Dominic Rech,1,1,1,1537,In Good Kompany
1,2,52,132,2319915,2363147,812849,NaN,10,1002,1,...,21,3110982,21377033,59,Dominic Rech,1,1,1,1537,In Good Kompany
2,3,53,185,1862688,1894075,531104,NaN,6,1008,1,...,20,3110982,21377033,59,Dominic Rech,1,1,1,1537,In Good Kompany
3,4,64,249,7650408,7666623,1455389,NaN,19,1013,1,...,8,3110982,21377033,59,Dominic Rech,1,1,1,1537,In Good Kompany
4,5,38,287,7572009,7616547,2126814,NaN,17,1011,1,...,7,3110982,21377033,59,Dominic Rech,1,1,1,1537,In Good Kompany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,21,42,1115,6574103,6596978,2685966,65.0,122,1017,2,...,4,5025791,39069622,44,Fin Whittington,16,16,16,1332,Is Your Motherwell
396,22,49,1160,5413499,5430275,2945224,55.0,71,1017,2,...,1,5025791,39069622,44,Fin Whittington,16,16,16,1332,Is Your Motherwell
397,23,46,1206,7330969,7387422,3430132,70.0,18,1019,0,...,2,5025791,39069622,44,Fin Whittington,16,16,16,1332,Is Your Motherwell
398,24,82,1288,1010944,1034053,2947329,10.0,6,1024,1,...,5,5025791,39069622,44,Fin Whittington,16,16,16,1332,Is Your Motherwell


In [13]:
def get_total_points_and_bench_points(duckdb_df):
    """
    This function returns a DataFrame with the total points and total points left on the bench for each player and team for the season.
    """
    return duckdb.query(
        """
        SELECT 
            player_name, 
            entry_name, 
            SUM(points_on_bench) AS bench_points, 
            SUM(event_points) AS total_points
        FROM 
            duckdb_df
        GROUP BY 
            player_name, 
            entry_name
    """
    ).to_df()
get_total_points_and_bench_points(duckdb_df)

,player_name,entry_name,bench_points,total_points
0,Alex Kitson,4-4-*******-2,106.0,1454.0
1,Mason Scott,FC Mase,148.0,1462.0
2,Liam Aylwin,Unbelievable Jeff!!,205.0,1466.0
3,Patrick Prendergast,Who Got The Assist,148.0,1460.0
4,Fin Whittington,Is Your Motherwell,123.0,1356.0
5,Daniel Brown,Benson and Hedges,217.0,1415.0
6,Prince Ehigiator,McCris-P FC,166.0,1415.0
7,Matt Harmer,Worrall Loada C*ap,91.0,1390.0
8,William Levick,Haalandaise Sauce,89.0,1387.0
9,Oliver Bignall,Biggie FC,180.0,1489.0


In [3]:


total_points_left_on_bench = duckdb.query("""
        SELECT 
            player_name, 
            COUNT(*) AS times_last_rank
        FROM (
            SELECT 
                event AS gameweek, 
                player_name
            FROM 
                duckdb_df
            WHERE 
                rank = (SELECT MAX(rank) AS FROM duckdb_df d2 WHERE d2.event = duckdb_df.event)
        ) subquery
        GROUP BY 
            player_name
        ORDER BY 
            times_last_rank DESC
        LIMIT 1
    """
    ).to_df()

BinderException: Binder Error: Referenced column "rank" not found in FROM clause!
Candidate bindings: "d2.bank", "d2.rank_sort", "d2.last_rank"

In [44]:
import duckdb

# Convert DataFrame to DuckDB
duckdb_df = duckdb.from_df(df)

# mydf = pd.DataFrame({'a' : [1, 2, 3]})

# print(duckdb.query("SELECT SUM(a) FROM mydf").to_df())

# Most points left on bench in a week
total_points_left_on_bench = duckdb.query("""
    SELECT player_name, entry_name, SUM(points_on_bench) AS bench_points
    FROM duckdb_df
    GROUP BY player_name, entry_name
    ORDER BY bench_points DESC
""").to_df()

most_points_left_on_bench_week = duckdb.query("""
    SELECT d.player_name, d.entry_name, d.points_on_bench AS most_points_left_on_bench, d.event
    FROM duckdb_df d
    JOIN (
        SELECT player_name, entry_name, MAX(points_on_bench) AS max_points
        FROM duckdb_df
        GROUP BY player_name, entry_name
    ) m ON d.player_name = m.player_name AND d.entry_name = m.entry_name AND d.points_on_bench = m.max_points
    ORDER BY most_points_left_on_bench DESC
""").to_df()

In [53]:
biggest_difference = duckdb.query("""
    SELECT a.event, a.player_name AS player1, a.entry_name AS entry1, a.event_points AS points1,
           b.player_name AS player2, b.entry_name AS entry2, b.event_points AS points2,
           ABS(a.event_points - b.event_points) AS difference
    FROM duckdb_df a, duckdb_df b
    WHERE a.event = b.event AND a.player_name != b.player_name
    ORDER BY difference DESC
    LIMIT 1
""").to_df()
biggest_difference

,event,player1,entry1,points1,player2,entry2,points2,difference
0,23,Jack Grimley,Cancelo Culture,44,William Levick,Haalandaise Sauce,110,66


In [6]:
# Most points left on bench in a week
most_points_left_on_bench = df['points_on_bench'].max()
print(f"Most points left on bench: {most_points_left_on_bench}")


Most points left on bench: 28


In [ ]:

# Worst week
worst_week = df['event_total_points'].idxmin()
print(f"Worst week: {worst_week}")


In [ ]:

# Biggest difference in points on a week from winner and loser
df_grouped = df.groupby('event')['event_total_points'].agg(['min', 'max'])
df_grouped['difference'] = df_grouped['max'] - df_grouped['min']
biggest_difference = df_grouped['difference'].max()
print(f"Biggest difference in points on a week from winner and loser: {biggest_difference}")